# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3248c66f10>
├── 'a' --> tensor([[-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326]])
└── 'x' --> <FastTreeValue 0x7f3248c66d00>
    └── 'c' --> tensor([[-1.5038,  1.2319, -1.3567,  1.3304],
                        [ 0.0822, -0.9802, -1.5036, -0.0740],
                        [-0.3451,  1.8868,  0.3188, -1.0186]])

In [4]:
t.a

tensor([[-2.3859, -0.3395,  0.0179],
        [ 0.8434, -0.7572, -0.6326]])

In [5]:
%timeit t.a

61.6 ns ± 0.0318 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3248c66f10>
├── 'a' --> tensor([[ 0.1982, -0.2652, -1.6321],
│                   [-1.4441, -1.2793,  0.8600]])
└── 'x' --> <FastTreeValue 0x7f3248c66d00>
    └── 'c' --> tensor([[-1.5038,  1.2319, -1.3567,  1.3304],
                        [ 0.0822, -0.9802, -1.5036, -0.0740],
                        [-0.3451,  1.8868,  0.3188, -1.0186]])

In [7]:
%timeit t.a = new_value

61.4 ns ± 0.0741 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326]]),
    x: Batch(
           c: tensor([[-1.5038,  1.2319, -1.3567,  1.3304],
                      [ 0.0822, -0.9802, -1.5036, -0.0740],
                      [-0.3451,  1.8868,  0.3188, -1.0186]]),
       ),
)

In [10]:
b.a

tensor([[-2.3859, -0.3395,  0.0179],
        [ 0.8434, -0.7572, -0.6326]])

In [11]:
%timeit b.a

57.3 ns ± 0.0214 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[0.0195, 0.9483, 0.3936],
               [0.8272, 0.8528, 0.1719]]),
    x: Batch(
           c: tensor([[-1.5038,  1.2319, -1.3567,  1.3304],
                      [ 0.0822, -0.9802, -1.5036, -0.0740],
                      [-0.3451,  1.8868,  0.3188, -1.0186]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 1.41 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 28.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 505 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 365 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f319b978d30>
├── 'a' --> tensor([[[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]],
│           
│                   [[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]],
│           
│                   [[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]],
│           
│                   [[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]],
│           
│                   [[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]],
│           
│                   [[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]],
│           
│                   [[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]],
│           
│                   [[-2.3859, -0.3395,  0.0179],
│                    [ 0.8434, -0.7572, -0.6326]]])
└── 'x' --> <FastTreeValue 0x7f319b978bb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 24.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f319b928b80>
├── 'a' --> tensor([[-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326],
│                   [-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326],
│                   [-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326],
│                   [-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326],
│                   [-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326],
│                   [-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326],
│                   [-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326],
│                   [-2.3859, -0.3395,  0.0179],
│                   [ 0.8434, -0.7572, -0.6326]])
└── 'x' --> <FastTreeValue 0x7f319b928c40>
    └── 'c' --> tensor([[-1.5038,  1.2319, -1.3567,  1.3304],
                        [ 0.0822, -0.9802, -1.5036, -0.0740],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 61.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]],
       
               [[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]],
       
               [[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]],
       
               [[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]],
       
               [[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]],
       
               [[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]],
       
               [[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]],
       
               [[-2.3859, -0.3395,  0.0179],
                [ 0.8434, -0.7572, -0.6326]]]),
    x: Batch(
           c: tensor([[[-1.5038,  1.2319, -1.3567,  1.3304],
                       [ 0.0822, -0.9802, -1.5036, -0.0740],
                       [-0.3451,  1.8868,  0.3188, -1.0186]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326],
               [-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326],
               [-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326],
               [-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326],
               [-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326],
               [-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326],
               [-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326],
               [-2.3859, -0.3395,  0.0179],
               [ 0.8434, -0.7572, -0.6326]]),
    x: Batch(
           c: tensor([[-1.5038,  1.2319, -1.3567,  1.3304],
                      [ 0.0822, -0.9802, -1.5036, -0.0740],
                      [-0.3451,  1.8868,  0.3188, -1.0186],
                      [-1.5038,  1.2319, -1.3567,  1.3304],
                      [ 0.0822, -

In [28]:
%timeit Batch.cat(batches)

141 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 817 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
